## 2. 크롤링 (문법 정리), method를 사용하여 표현해 주세요.

### 셀레늄을 사용하여 웹 페이지를 어떻게 로드할 수 있나요?

In [ ]:
## vscode 기준
# 셀레늄 설치
%pip install selenium
# webdriver-manager 설치
%pip install webdriver-manager

In [4]:
# import
from selenium import webdriver

browser = webdriver.Chrome()

### 셀레늄으로 웹 페이지에서 특정 요소를 어떻게 찾을 수 있나요? 요소를 찾을 수 있는 함수는 어떤 것들이 있나요? ex) ID, XPATH…

In [ ]:
# find_element 또는 find_elements 함수를 사용한다.
from selenium.webdriver.common.by import By
# By import - By를 통해 어떤 것을 기준으로 특정 요소를 찾을 것인지 선택할 수 있다.

browser.find_element(By.CLASS_NAME, 'class_name')
browser.find_element(By.XPATH, 'element_right_click-Copy-Copy_XPath')
browser.find_element(By.ID, 'id')


### 셀레늄으로 찾은 요소의 텍스트를 가져오려면 어떻게 해야 하나요?


In [ ]:
browser.find_element(By.ID, 'id').text

### 셀레늄에서 특정 텍스트를 입력하고 엔터 키를 누르는 방법은 무엇인가요?

In [ ]:
from selenium.webdriver.common.keys import Keys

element = browser.find_element(By.ID, 'id')
element.send_keys("입력값")
element.send_keys(Keys.ENTER)

### 셀레늄으로 찾은 요소의 속성값을 가져오려면 어떻게 해야 하나요?

In [ ]:
browser.find_element(By.ID).get_attribute('attribute')

### 셀레늄으로 웹 페이지를 스크롤하는 방법은 무엇인가요?

In [ ]:
# chatGPT에게 물어보고 문서 찾아보기
# with javascript
## https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.execute_script
browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", element)

# with ActionChains
## https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.action_chains
from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(browser)
# 페이지 스크롤 다운
actions.send_keys(Keys.END).perform()
# 특정 요소 스크롤 다운
## 이 방법으로는 원하는 결과를 보진 못 함...
actions.move_to_element(element).click_and_hold().move_by_offset(0, 100).release().perform()

### 셀레늄으로 텍스트를 입력하는 방법은 무엇인가요?

In [ ]:
browser.find_element(By.ID, '').send_keys("입력값")

### 셀레늄으로 웹 페이지에서 이전 페이지로 이동하는 방법은 무엇인가요?

In [ ]:
browser.back()

### 셀레늄으로 웹 페이지에서 다음 페이지로 이동하는 방법은 무엇인가요?

In [ ]:
browser.forward()

### 셀레늄의 Implicit Waits와 Explicit Waits의 차이점은 무엇인가요?

TODO:

### 구글 플레이 스토어에서 앱 검색 후 댓글을 수집하는 코드를 작성하시오.

- 조건1: 내가 입력한 검색어를 기반으로 첫 번째 앱을 선택합니다.
- 조건2: 댓글 수집이 100개 이상 수집되어야 합니다. (댓글 갯수가 100미만인 앱은 예외)
- 조건3: 댓글에서 아래의 정보를 가져오세요.
    - 작성자명, 별표 갯수, 작성일, 작성 내용
- 조건4: 크롤링한 데이터를 오늘 날짜를 기준으로 엑셀에 저장하시오. (파일명: 2023-04-18)

In [240]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd


browser = webdriver.Chrome()

# 1.
keyword = input("검색할 앱 이름을 입력하세요!")
url = f'https://play.google.com/store/search?q={keyword}&c=apps&hl=en-KR'
browser.get(url)
# 첫 번째 앱 선택
# ## class="Qfxief"
first_app_link = browser.find_element(By.CLASS_NAME, 'Qfxief').get_attribute('href')
browser.get(first_app_link)

# 2.
## see all reviews 버튼 누르기
## class="VfPpkd-J1Ukfc-LhBDec"
xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div/div[2]/div/div[1]/div[1]/c-wiz[5]/section/div/div[2]/div[5]/div/div/button'
# xpath = '//*[@id="ow71"]/section/div/div[2]/div[5]/div/div/button'
browser.find_element(By.XPATH, xpath).click()

time.sleep(3)

## 모달 찾아서 스크롤하기
modal_xpath = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]'
modal = browser.find_element(By.XPATH, modal_xpath)
##FIXME: 어떻게 끝까지 스크롤 해야 할까
for i in range(30):
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", modal)
    time.sleep(0.5)

# 댓글 컨테이너:
## class="RHo1pe"
### 100 개
comment_container_list = browser.find_elements(By.CLASS_NAME, 'RHo1pe')
if len(comment_container_list) < 100:
    print("댓글수가 100개 미만입니다.")
else:
    # 3.
    comment_list = []

    for comment in comment_container_list:
        ## 작성자명
        ## class="X5PpBb"
        user_name = comment.find_element(By.CLASS_NAME, 'X5PpBb').text
        # print("user_name: ", user_name)

        ## 별표 갯수
        ## class="iXRFPc"
        ## aria-label="Rated 1 stars out of five stars"
        rating = comment.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute("aria-label")
        stars = rating.split(" ")[1]
        # print("rating: ", rating)
        # print("stars: ", stars)

        ## 작성일
        ## class="bp9Aid"
        comment_date = comment.find_element(By.CLASS_NAME, 'bp9Aid').text
        # print("comment_date: ", comment_date)

        ## 작성 내용
        ## class="h3YV2d"
        contents = comment.find_element(By.CLASS_NAME, 'h3YV2d').text
        # print("contents: ", contents)

        comment_list.append({
            '작성자명': user_name,
            '별표 갯수': stars,
            '작성일': comment_date,
            '작성 내용': contents
        })

        # 4.
        ## 작성일 날짜
        ## https://docs.python.org/3/library/time.html#time.struct_time

        current_time = time.localtime()
        year = str(current_time[0])
        month = str(current_time[1])
        day = str(current_time[2])

        def format_date(date):
            formatted_date = date
            if len(date) < 2:
                formatted_date = f"0{date}"
            return formatted_date

        month = format_date(month)
        day = format_date(day)

        file_name = f"{year}-{month}-{day}"

        ## 엑셀로 저장
        df = pd.DataFrame(comment_list)
        df.to_excel(f"{file_name}.xlsx")

print("done!")


done!


In [199]:
# 3.
# 댓글 컨테이너:
## class="RHo1pe"
comment_container_list = browser.find_elements(By.CLASS_NAME, 'RHo1pe')
comment_list = []

for comment in comment_container_list:
    ## 작성자명
    ## class="X5PpBb"
    user_name = comment.find_element(By.CLASS_NAME, 'X5PpBb').text
    # print("user_name: ", user_name)

    ## 별표 갯수
    ## class="iXRFPc"
    ## aria-label="Rated 1 stars out of five stars"
    rating = comment.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute("aria-label")
    stars = rating.split(" ")[1]
    # print("rating: ", rating)
    # print("stars: ", stars)

    ## 작성일
    ## class="bp9Aid"
    comment_date = comment.find_element(By.CLASS_NAME, 'bp9Aid').text
    # print("comment_date: ", comment_date)

    ## 작성 내용
    ## class="h3YV2d"
    contents = comment.find_element(By.CLASS_NAME, 'h3YV2d').text
    # print("contents: ", contents)

    comment_list.append({
        '작성자명': user_name,
        '별표 갯수': stars,
        '작성일': comment_date,
        '작성 내용': contents
    })

print(len(comment_list))
comment_list

46


[{'작성자명': 'Mike C',
  '별표 갯수': '3',
  '작성일': '9 January 2023',
  '작성 내용': 'The entire interface is weird. The app expects you to scan in the entry ticket qr code. They should make the app able to upload qr codes instead of scanning. The idea for queueing in advance is great. It has an inbuilt gps, but the map is still in Korean which defeats the purpose. Someone needs to review the English used in the app. Overall an difficult to use app and not internationally tourist friendly.'},
 {'작성자명': 'Josie',
  '별표 갯수': '5',
  '작성일': '4 November 2022',
  '작성 내용': "I was suprised that the app was in English when it finished downloading but it made navigating much easier. I purchased my tickets through Klook and was able to scan the QR codes and register them into the app. I primarily used the app to check ride wait times, nearby restaurants and washrooms and lockers. I can't say much about their version of a Fast Pass and if it can be purchased/used on the app but it did what I needed it for whe

In [104]:
import time
## https://docs.python.org/3/library/time.html#time.struct_time

current_time = time.localtime()
year = str(current_time[0])
month = str(current_time[1])
day = str(current_time[2])

def format_date(date):
    formatted_date = date
    if len(date) < 2:
        formatted_date = f"0{date}"
    return formatted_date

month = format_date(month)
day = format_date(day)

file_name = f"{year}-{month}-{day}"

print(file_name)


2024-04-08


In [105]:
## 엑셀로 저장
import pandas as pd
df = pd.DataFrame(comment_list)
df

,작성자명,별표 갯수,작성일,작성 내용
0,Mike C,3,9 January 2023,The entire interface is weird. The app expects...
1,Josie,5,4 November 2022,I was suprised that the app was in English whe...
2,Lim Dao Wei,1,18 March 2024,Don't bother using this app if you are a forei...
3,sam ang,1,16 March 2024,Cannot log in or register through email
4,Grace Pan,1,13 May 2022,Don't bother coming to Everland if you are a f...
5,Anton Dianov,1,10 May 2023,"Extremely bad application, permanently fails t..."
6,Faith Wright,1,9 February 2024,YOU DON'T NEED THIS APP. YOU DON'T NEED BAO HA...
7,Joshua Felice,1,15 May 2021,The app and the Amusement Park ARE NOT foreign...
8,A Google user,2,19 April 2019,this app is now available in english. but ther...
9,izzy mendoza,1,29 May 2022,My friend bought me a pass and I cannot regist...


In [106]:
## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
# df.to_excel()
df.to_excel(f"{file_name}.xlsx")